In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/piggyjehong/yolov3.git

Cloning into 'yolov3'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 137 (delta 76), reused 3 (delta 0), pack-reused 0
Receiving objects: 100% (137/137), 1.33 MiB | 5.24 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [ ]:
%cd /content/yolov3/yolov3-archive
!pip install -r requirements.txt

In [ ]:
%cp -r /content/drive/MyDrive/voc.zip /content/yolov3/yolov3-archive

In [ ]:
!unzip /content/yolov3/yolov3-archive/voc.zip

In [ ]:
%cp -r /content/yolov3/yolov3-archive/voc/images /content/yolov3/yolov3-archive/data
%cp -r /content/yolov3/yolov3-archive/voc/labels /content/yolov3/yolov3-archive/data
%cp -r /content/yolov3/yolov3-archive/voc/2007_test.txt /content/yolov3/yolov3-archive/data
%cp -r /content/yolov3/yolov3-archive/voc/2007_train.txt /content/yolov3/yolov3-archive/data
%cp -r /content/yolov3/yolov3-archive/voc/template.data /content/yolov3/yolov3-archive/data
%cp -r /content/yolov3/yolov3-archive/voc/template.names /content/yolov3/yolov3-archive/data

In [ ]:
#### Modify template.data
'''
classes=3 
train=/content/yolov3/yolov3-archive/data/2007_train.txt
valid=/content/yolov3/yolov3-archive/data/2007_test.txt
names=/content/yolov3/yolov3-archive/data/template.names
backup=/content/drive/MyDrive/backup
eval=coco

'''

In [ ]:
def alter(file,old_str,new_str):
    """
    替换文件中的字符串
    :param file:文件名
    :param old_str:就字符串
    :param new_str:新字符串
    :return:
    
    """
    file_data = ""
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            if old_str in line:
                line = line.replace(old_str,new_str)
            file_data += line
    with open(file,"w",encoding="utf-8") as f:
        f.write(file_data)

In [ ]:
alter("/content/yolov3/yolov3-archive/data/2007_test.txt", "./data/images/test2014/", "/content/yolov3/yolov3-archive/data/images/val2014/")
alter("/content/yolov3/yolov3-archive/data/2007_train.txt", "./data/images/train2014/", "/content/yolov3/yolov3-archive/data/images/train2014/")


In [ ]:
%mkdir weights
%cp -r /content/drive/MyDrive/yolov3_spp/yolov3-tiny.pt /content/yolov3/yolov3-archive/weights

In [ ]:
!python3 train.py --device 0,1 --epochs 600 --data /content/yolov3/yolov3-archive/data/template.data --cfg /content/yolov3/yolov3-archive/cfg/yolov3-tiny-3cls.cfg  --weights /content/yolov3/yolov3-archive/weights/yolov3-tiny.pt 

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='/content/yolov3/yolov3-archive/cfg/yolov3-tiny-3cls.cfg', data='/content/yolov3/yolov3-archive/data/template.data', device='0,1', epochs=600, evolve=False, freeze_layers=False, img_size=[320, 640], multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='/content/yolov3/yolov3-archive/weights/yolov3-tiny.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/
2021-05-12 03:56:05.798310: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Model Summary: 37 layers, 8.6745e+06 parameters, 8.6745e+06 gradients
Optimizer groups: 13 .bias, 13 Conv2d.weight, 11 other
Reading image shapes: 100% 738/7

In [ ]:
!python3 test.py --device 0,1 --data /content/yolov3/yolov3-archive/data/template.data --cfg /content/yolov3/yolov3-archive/cfg/yolov3-tiny-3cls.cfg  --weights /content/yolov3/yolov3-archive/weights/last.pt 

Namespace(augment=False, batch_size=16, cfg='/content/yolov3/yolov3-archive/cfg/yolov3-tiny-3cls.cfg', conf_thres=0.001, data='/content/yolov3/yolov3-archive/data/template.data', device='0,1', img_size=512, iou_thres=0.6, save_json=False, single_cls=False, task='test', weights='/content/yolov3/yolov3-archive/weights/last.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Model Summary: 37 layers, 8.6745e+06 parameters, 8.6745e+06 gradients
Fusing layers...
Model Summary: 26 layers, 8.67131e+06 parameters, 8.67131e+06 gradients
Caching labels /content/yolov3/yolov3-archive/data/2007_test.txt (185 found, 0 missing, 0 empty, 0 duplicate, for 185 images): 100% 185/185 [00:00<00:00, 4608.36it/s]
               Class    Images   Targets         P         R   mAP@0.5        F1: 100% 12/12 [00:02<00:00,  5.54it/s]
                 all       185       816     0.755     0.702     0.724     0.715
           with_mask       185       660     0.817    

In [ ]:
%cp -r /content/yolov3/yolov3-archive/weights/best.pt /content/drive/MyDrive/result/tiny-600/
%cp -r /content/yolov3/yolov3-archive/weights/last.pt /content/drive/MyDrive/result/tiny-600/